## Column-stack time series

In [1]:
import os
import pandas as pd
from glob import glob
from config import processed_data_dir

seed=2019

train_files = glob(os.path.join(processed_data_dir, 'baseline/train/*.csv'))
test_files = glob(os.path.join(processed_data_dir, 'baseline/test/*.csv'))

In [2]:
from tqdm import tqdm
def column_stack(file_list):
    target_col = 'Crop_Id_Ne'
    dfs = []
    for i,fpath in tqdm(enumerate(file_list)):
        ts = os.path.basename(fpath).split('.')[0]

        df = pd.read_csv(fpath, index_col=0)

        new_columns = [f'{c}_{ts}' for c in list(df) if c != target_col]

        if i > 0 and target_col in list(df):
            df.drop(target_col, axis=1, inplace=True)

        elif target_col in list(df):
            new_columns.append(target_col)

        df.columns = new_columns

        dfs.append(df)
        
    return pd.concat(dfs,axis=1)


In [3]:
train_data = column_stack(train_files)
test_data = column_stack(test_files)

11it [00:00, 31.44it/s]
11it [00:00, 59.25it/s]


In [ ]:
y = train_data.Crop_Id_Ne
X = train_data.drop('Crop_Id_Ne', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.75, test_size=0.25, 
                                                    random_state=seed)

tpot = TPOTClassifier(
    generations=5, 
    population_size=20, 
    verbosity=2, 
    n_jobs=-1,  
    scoring='accuracy',
    cv=5,
    random_state=seed, 
    use_dask=True, 
    periodic_checkpoint_folder='checkpoints'
)

tpot.fit(X_train, y_train)

Bombing out because too much compute...